In [ ]:
import os
import json
import pandas as pd
import traceback
import warnings
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain_core.output_parsers import JsonOutputParser
# from langchain.callbacks import get_openai_callback
import PyPDF2

warnings.filterwarnings("ignore")

In [3]:
llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-V3.1",
)

# Here, llm is not a normal text LLM, it’s a chat LLM as deepseek-ai/DeepSeek-V3.1 doesnt support text but only conversation— so LLMChain doesn’t really know how to talk to it. That’s why LangChain has ChatHuggingFace, which wraps HuggingFaceEndpoint and converts it into something LLMChain can understand.
chat_model = ChatHuggingFace(llm=llm)

In [4]:
output_response = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

output_response_json = json.dumps(output_response)

In [5]:
quiz_generation_template="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students with difficulty level {level}. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "level", "response_json"],
    template=quiz_generation_template
    )
quiz_chain=LLMChain(llm=chat_model, prompt=quiz_generation_prompt, output_key="quiz", output_parser= JsonOutputParser(), verbose=True)

In [6]:
quiz_evaluation_template="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=quiz_evaluation_template)
review_chain=LLMChain(llm=chat_model, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [7]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "level", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [ ]:
file_path=r"D:\Gen Ai\MCQGenerator\topic.txt"

with open(file_path, 'r') as file:
    topic = file.read()

Anatomically modern humans first arrived on the Indian subcontinent between 73,000 and 55,000 years ago.[1] The earliest known human remains in South Asia date to 30,000 years ago. Sedentariness began in South Asia around 7000 BCE; by 4500 BCE, settled life had spread,[2] and gradually evolved into the Indus Valley Civilisation, one of three early cradles of civilisation in the Old World,[3][4] which flourished between 2500 BCE and 1900 BCE in present-day Pakistan and north-western India. Early in the second millennium BCE, persistent drought caused the population of the Indus Valley to scatter from large urban centres to villages. Indo-Aryan tribes moved into the Punjab from Central Asia in several waves of migration. The Vedic Period of the Vedic people in northern India (1500–500 BCE) was marked by the composition of their extensive collections of hymns (Vedas). The social structure was loosely stratified via the varna system, incorporated into the highly evolved present-day Jāti sy

In [ ]:
number=5 
subject="History"
level="simple"
response=generate_evaluate_chain(
    {
        "text": topic,
        "number": number,
        "subject":subject,
        "level": level,
        "response_json": output_response_json,
    }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Anatomically modern humans first arrived on the Indian subcontinent between 73,000 and 55,000 years ago.[1] The earliest known human remains in South Asia date to 30,000 years ago. Sedentariness began in South Asia around 7000 BCE; by 4500 BCE, settled life had spread,[2] and gradually evolved into the Indus Valley Civilisation, one of three early cradles of civilisation in the Old World,[3][4] which flourished between 2500 BCE and 1900 BCE in present-day Pakistan and north-western India. Early in the second millennium BCE, persistent drought caused the population of the Indus Valley to scatter from large urban centres to villages. Indo-Aryan tribes moved into the Punjab from Central Asia in several waves of migration. The Vedic Period of the Vedic people in northern India (1500–500 BCE) was marked by the composition of their extensive collections of hymns (Vedas). The social str

In [11]:
print("QUIZ DICT:", response["quiz"])
print("REVIEW:", response["review"])

QUIZ DICT: {'1': {'mcq': 'When did sedentariness (settled life) begin in South Asia?', 'options': {'a': 'Around 30,000 years ago', 'b': 'Around 7000 BCE', 'c': 'Between 73,000 and 55,000 years ago', 'd': 'Around 2500 BCE'}, 'correct': 'b'}, '2': {'mcq': 'Which empire, established by Chandragupta Maurya, was the first great empire in ancient India?', 'options': {'a': 'The Gupta Empire', 'b': 'The Shunga Empire', 'c': 'The Maurya Empire', 'd': 'The Nanda Empire'}, 'correct': 'c'}, '3': {'mcq': 'What was the Delhi Sultanate?', 'options': {'a': 'A Hindu empire that ruled southern India', 'b': 'A dynasty established by the Mughals', 'c': 'A sultanate ruled by various Turkic and Afghan dynasties in northern India', 'd': 'A wealthy independent sultanate in Bengal'}, 'correct': 'c'}, '4': {'mcq': 'What event led to the direct rule of India by the British Crown (the British Raj)?', 'options': {'a': 'World War I', 'b': 'The invasions of Timur', 'c': 'The Indian Rebellion of 1857', 'd': 'The rise

In [15]:
quiz_response = response["quiz"]

quiz_table_data = []
for key, value in quiz_response.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

quiz_table_data_df = pd.DataFrame(quiz_table_data)
quiz_table_data_df

,MCQ,Choices,Correct
0,When did sedentariness (settled life) begin in...,"a: Around 30,000 years ago | b: Around 7000 BC...",b
1,"Which empire, established by Chandragupta Maur...",a: The Gupta Empire | b: The Shunga Empire | c...,c
2,What was the Delhi Sultanate?,a: A Hindu empire that ruled southern India | ...,c
3,What event led to the direct rule of India by ...,a: World War I | b: The invasions of Timur | c...,c
4,Which two new ascetic movements arose in India...,a: Hinduism and Islam | b: Jainism and Buddhis...,b


In [16]:
quiz_table_data_df.to_csv("quiz_history.csv", index=False)